In [2]:
import rasterio
import geopandas as gpd
import numpy as np
from skimage.graph.mcp import MCP_Geometric
from onsstove.raster import *

In [11]:
# This map represents the travel speed from this allocation process, expressed in units of minutes 
# required to travel one metre (Raster).
friction = r"output\supply\Biomass\Biomass - friction - reprojected.tif"
# Startpoints (Raster)
forest_cover = r"output\supply\Biomass\Biomass - reprojected.tif"
# Potential restriction areas (Polygon)
restrictions = r"output\supply\Biomass\protected.shp"
restricted = r"output\supply\Biomass\restricted_area.tif"
#new friction based on start points (Raster)
travel_time = r"output\supply\Biomass\travel_time - test.tif"

In [ ]:
#Generated a restriction mask
mask_raster(forest_cover, restrictions, restricted, nodata=0, compression='NONE')
new_restrictions, new_restrictions_meta = align_raster(friction, restricted, method='nearest', compression='NONE')

In [4]:
#Allign the forest raster with the friction raster
new_forest, new_forest_meta = align_raster(friction, forest_cover, method='nearest', compression='NONE')
# new_forest = new_forest+new_restrictions
new_forest[new_forest == 1] = 0
new_forest[new_forest != 0] = 1
rows, cols = np.where(new_forest == 0)

In [5]:
# Transform friction raster to km and hour instead of minutes and meter
with rasterio.open(friction) as fric:
    fric_meta = fric.meta
    fric_val = fric.read(1)
    fric_val = fric_val*1000/60
    fric_val[np.isnan(fric_val)] = float('inf')

In [6]:
# Create a new friction map by multiplying the firction map with the new forest cover map
new_fric = fric_val*new_forest

<ipython-input-6-1efa2ec227d7>:2: RuntimeWarning: invalid value encountered in multiply
  new_fric = fric_val*new_forest


In [7]:
# Minimum cost path betweenn all nodes
mcp = MCP_Geometric(new_fric, fully_connected=True)

In [8]:
#Adding statvalues
pointlist = []
i = 0
for row in rows:
    positiions = [rows[i],cols[i]]
    pointlist.append(positiions)
    i = i+1

In [9]:
#minimum cost
cumulative_costs, traceback = mcp.find_costs(starts=pointlist)
cumulative_costs[np.where(cumulative_costs==float('inf'))] = np.nan

In [14]:
new_dataset = rasterio.open(travel_time, 'w', driver='GTiff',
                            height = fric_val.shape[0], width = fric_val.shape[1],
                            count=1, dtype="float64",
                            crs=fric_meta['crs'], transform = fric_meta['transform'])

new_dataset.write(cumulative_costs, 1)
new_dataset.close()

In [18]:
pointlist[0]

[0, 0]

In [20]:
mcp.traceback(pointlist[100])

[(0, 100)]